# Intelligence artificielle et raisonnement TP1

### Code base

On a implementer la class `Taquin`, une package `algorithms` qui contient les methodes demander.

Le contenu de `algorithms` package est:
- `BaseAlgorithm`: C'est une class de base pour les solutions
- `AStar`: A* Algorithm
- `Dfs`: Depth First Search
- `IterativeDfs`: Iterative Depth First Search

In [28]:
from taquin import Taquin
from algorithms import Dfs, IterativeDfs, AStar  # noqa: F401

ImportError: cannot import name 'BaseAlgorithm' from 'algorithms' (c:\Users\saida\OneDrive\Documents\Github\TP-AI-GL4\algorithms\__init__.py)

### Objectif: Optimisation de calcule de fonction d'heuristique

Cette optimisation depand de la fonction de calcule d'heuristique choisit.

Pour simplification, on a utiliser cette heuristique:

In [2]:
def h1(board: Taquin) -> int:
  state = board.get_state()
  goal = board.get_solution()
  cost = 0
  for i in range(board.size):
    for j in range(board.size):
      if goal[i][j] != state[i][j]:
        cost += 1
  
  return cost

On doit tout d'abords calculer la complexité de cette fonctions. C'est O(N<sup>2</sup>) avec `N` la taille de board.

Pour des grand valeurs de `N`, le calcule de cout peut prendre beaucoup du temps.

### Principe de solution

On prend cette example de board:

<p align="center">
  <img src="./img/board1.png">
</p>

In [3]:
state = [
  [7,2,4],
  [6,5,0],
  [8,3,1],
]

board = Taquin(3)
board.set_state(state)
print(board)

  7  2  4
  6  5  0
  8  3  1



Calculons le cout de cette etat:

In [4]:
current_cost = h1(board)
current_cost

9

Si on prend un movement aleatoire pour creer une nouvelle etat, par example un deplacement avec `5`, on se trouvera avec cette nouvelle etat: 

<p align="center">
  <img src="./img/board2.png">
</p>

In [5]:
new_board = board.move([0, -1])
print(new_board)
h1(new_board)

  7  2  4
  6  0  5
  8  3  1



8

La case `5` est maintenant en position correct, donc le cout est diminuer de 1. Mais c'est calculer à l'aide d'un parcours complete de board!

Notre solution consiste a calculer le nouveau cout avec une complexité `O(1)`.

Pour le faire, nous devons savoir exactement quels changements ont eu lieu aprés chanque mouvement et quels sont les effets de ces changements sur le cout.

Commencant par les changements, notre mouvement consiste toujour a échanger (swap) 2 tiles, le `Tile 0` et un autre qu'on choisit.

Pour notre example, la fonction heuristique depend de nombre d'elements placer correctement sur la board. Donc, on doit voir quels sont les effets de notre mouvement sur cette condition.

Puisque nous somme entrain de échanger une `tile X` avec la `tile 0`, seulement ces 2 tiles seront affecté par notre deplacement.

On a 2 tiles, chaqu'une a 2 etats:
- Mal placé
- Bien placé

Donc, on peut avoir 2<sup>2</sup>=4 possibilité pour chaque tile.

On peut conclur ces situations:
* `Tile X`:
  - `TXP0`: A l'etat current est mal placé. Dans la nouvelle etat, elle est mal placé.
  - `TXP1`: A l'etat current est mal placé. Dans la nouvelle etat, elle est bien placé.
  - `TXP2`: A l'etat current est bien placé. Dans la nouvelle etat, elle est mal placé.
  - `TXP3`: A l'etat current est bien placé. Dans la nouvelle etat, elle est bien placé.


* `Tile 0`:
  - `T0P0`: A l'etat current est mal placé. Dans la nouvelle etat, elle est mal placé.
  - `T0P1`: A l'etat current est mal placé. Dans la nouvelle etat, elle est bien placé.
  - `T0P2`: A l'etat current est bien placé. Dans la nouvelle etat, elle est mal placé.
  - `T0P3`: A l'etat current est bien placé. Dans la nouvelle etat, elle est bien placé.

Maintenant, on doit voir l'effet de ces changements sur notre fonction heuristique. Ces changements depend de la fonction heuristique et doit etre reconsiderer pour utiliser une nouvelle fonction:

* `Tile X`
  - `TXP0`: Pas de changement sur le cout. (`0`)
  - `TXP1`: Diminuer le cout par 1. (`-1`)
  - `TXP2`: Incrementer le cout par 1. (`+1`)
  - `TXP3`: Comme chaque tile a une seul position correct, cette situation est impossible.

C'est les memes situations et effets pour la `Tile 0`.

Donc, aprés chaque movement, on peut calculer la nouvelle cout en fonction de cout précédent. Dans notre example, on a:
- `T0P0` est correct pour `Tile 0` (`0`)
- `TXP1` est correct pour `Tile X` (`-1`)

La difference total `d=-1`, donc `nouveau_cout = cout_prec + d = 9 - 1 = 8`

### Implementation

On prend la method `A*` comme example. A l'initialisation, on doit calculer le cout de taquin initiale. Le cout donnée par la fonction heuristique est sauvegarder dans l'instance de `Taquin` qu'il est associé.

```py
class AStar(BaseAlgorithm):
    ...
    
    def __init__(self, taquin: Taquin, quiz_cost: bool = False):
        self.goal = taquin._goal
        self.quiz_cost = quiz_cost
        
        taquin.f = 0
        taquin.g = 0
        taquin.h = self.get_cost(taquin)
        
        self.taquin: Taquin = taquin
        
        ...
    
    ...
```

Le parametre `quiz_cost` est utilisé pour specifier est ce qu'on doit recalculer le cout à l'aide de fonction heuristique ou à l'aide de notre algorithm.

```py
    def solve(self, timeout: int = 10):
            ...
            children: list[Taquin] = []
            for direction in Taquin.validDirections:
                child = current.move(direction)
                if child is not None:
                    self.update_cost(current, child)
                    children.append(child)
            ...
```

Lors de generation des mouvements possibles, on fait appelle à `update_cost` qui est responsable de calculer et metre a jour le cout de nouvelle instance `child`.

```py
    ...
    def update_cost(self, prev: Taquin, next: Taquin) -> None:
        # Tile 0 will always be moved.
        board_size = next.size
        prev_T0_x, prev_T0_y = prev.get_tile(0)
        next_T0_x, next_T0_y = next.get_tile(0) # This is the index of the tile we swapping with
        tile_value = prev._state[next_T0_x][next_T0_y]
        correct_x, correct_y = tile_value//board_size, tile_value % board_size
        
        diff = 0
        if prev_T0_x == prev_T0_y == 0:
            # Was correct, now it's wrong
            diff += 1
        
        if next_T0_x == next_T0_y == 0:
            # Was wrong, now it's correct
            diff -= 1
        
        if correct_x == prev_T0_x and correct_y == prev_T0_y:
            # Now it's correct, 100% it was wrong
            diff -= 1
        else:
            # Now, it's wrong
            if correct_x == next_T0_x and correct_y == next_T0_y:
                # It was correct! We made it worst
                diff += 1
            # If it was wrong, nothing changes
        
        next.h = prev.h + diff

```

Basé sur les conditions qu'on a déduit, on a implementer cette method pour calculer la difference entre le cout precedant et le cout de nouvelle etat. Puis, on mettre a jour l'attribut `h` de la nouvelle instance `next`.